In [24]:

import os
import sys
from pyspark.sql.functions import *
import findspark
findspark.init('C:/Spark/spark-3.4.2-bin-hadoop3/')

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable



from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "10g").master('local[*]').appName('lab_spark')\
    .config('spark.jars.packages', 'org.apache.spark:spark-avro_2.12:3.4.2')\
    .config('spark.jars', 'C:\\Spark\\spark-3.4.2-bin-hadoop3\\jars\\postgresql-42.7.3.jar')\
    .getOrCreate()

In [52]:
data = spark.read\
    .option('delimiter',';')\
    .csv(r'task1\part-00000-23875efd-ccbb-4f51-915d-6d8c984185c4-c000.csv')\
    .withColumnRenamed('_c0','city')\
    .withColumnRenamed('_c1','country')\
    .withColumnRenamed('_c2','facility_name')\
    .withColumnRenamed('_c3','lat')\
    .withColumnRenamed('_c4','lng')\
    .withColumnRenamed('_c5','order_id')\
    .withColumnRenamed('_c6','time')\
    .withColumnRenamed('_c7','food')\
    .withColumnRenamed('_c8', 'price')


In [53]:
data.show()

+-----------+-------+--------------------+----------+----------+--------+-------------------+--------------+------+
|       city|country|       facility_name|       lat|       lng|order_id|               time|          food| price|
+-----------+-------+--------------------+----------+----------+--------+-------------------+--------------+------+
|Novosibirsk| Russia|       #СИБИРЬСИБИРЬ|  -999.999|  -999.999|   73231|2025-01-01 03:06:34|           Tea|  null|
|       null| Russia|       #СИБИРЬСИБИРЬ|  -999.999|82.9049737|   73231|2025-01-01 03:06:34|          null|134.90|
|Novosibirsk| Russia|       #СИБИРЬСИБИРЬ|  -999.999|  -999.999|   73231|2025-01-01 03:06:34|          null|  null|
|Novosibirsk| Russia|       #СИБИРЬСИБИРЬ|55.0295233|82.9049737|   73231|2025-01-01 03:06:34|      Red fish|978.71|
|Novosibirsk| Russia|       #СИБИРЬСИБИРЬ|  -999.999|82.9049737|   73231|               null|  French fries|150.00|
|Novosibirsk| Russia|                null|  -999.999|82.9049737|   73231

In [55]:
clear = data.filter((data.city.isNotNull()) & 
                    (data.country.isNotNull()) & 
                    (data.facility_name.isNotNull()) & 
                    (data.food.isNotNull()) & 
                    (data.price.isNotNull()) & 
                    (data.time.isNotNull()) & 
                    (data.lng != -999.999) & 
                    (data.lat != -999.999))

In [56]:
dirty = data.filter((data.city.isNull()) | 
                    (data.country.isNull()) | 
                    (data.facility_name.isNull()) | 
                    (data.food.isNull()) | 
                    (data.price.isNull()) |
                    (data.time.isNull()) |
                    (data.lng == -999.999) |
                    (data.lat == -999.999))

In [57]:
clear = clear.withColumn("date", col("time").substr(1, 10))
clear = clear.withColumn("time", col("time").substr(12, 8))

In [58]:
clear.show()

+----------------+-------+--------------------+----------+-----------+--------+--------+-------------------+-------+----------+
|            city|country|       facility_name|       lat|        lng|order_id|    time|               food|  price|      date|
+----------------+-------+--------------------+----------+-----------+--------+--------+-------------------+-------+----------+
|     Novosibirsk| Russia|       #СИБИРЬСИБИРЬ|55.0295233| 82.9049737|   73231|03:06:34|           Red fish| 978.71|2025-01-01|
|     Vladivostok| Russia|TOKYO Kawaii на С...| 43.119274| 131.882176|   23612|03:13:59|           Red fish| 978.71|2025-01-01|
|       Krasnodar| Russia|        Кафе Винсент| 45.020957| 38.9700078|   73416|03:20:03|         White wine| 575.40|2025-01-01|
|     Vladivostok| Russia|         Супра Меоре| 43.101581|131.9051689|   64036|03:33:10|         White wine| 575.40|2025-01-01|
|     Vladivostok| Russia|         Супра Меоре| 43.101581|131.9051689|   64036|03:33:10|              Pa

In [59]:
clear.write.partitionBy("date", "city") \
        .format("avro") \
        .mode("overwrite") \
        .save("orders_avro")

clear.write.partitionBy("date", "city") \
        .format("parquet") \
        .mode("overwrite") \
        .save("orders_parquet")